This notebook is used to save the tidal parameters calculated from the model output to a csv file in order to avoid having to load all the files run.

In [241]:
import os
import glob
import datetime
import matplotlib.pylab as plt
import matplotlib.ticker as ticker
from matplotlib.patches import Ellipse
import numpy as np
from IPython.display import display, Math, Latex
import csv
import pandas as pd
import re

import netCDF4 as nc
from scipy.optimize import curve_fit
from salishsea_tools import (viz_tools,tidetools, nc_tools)
from salishsea_tools.nowcast import (research_VENUS, analyze)

%matplotlib inline


#Hourly Data
##@ VENUS nodes
The functions below will facilitate loading the hourly data and writting the tidal parameters to a csv file.

In [184]:
def loadparam_h(to,tf):
    filesu=analyze.get_filenames(to,tf, '1h', 'grid_U', path)
    filesv=analyze.get_filenames(to,tf,'1h', 'grid_V', path)

    sites=research_VENUS.SITES
    i_c=sites['VENUS']['Central']['i']
    i_e=sites['VENUS']['East']['i']
    j_c=sites['VENUS']['Central']['j']
    j_e=sites['VENUS']['East']['j']

    u_u_c, time = analyze.combine_files(filesu, 'vozocrtx','None',[j_c-1, j_c], [i_c-1,i_c])
    v_v_c, timec = analyze.combine_files(filesv, 'vomecrty','None',[j_c-1, j_c], [i_c-1,i_c])
    time_c = tidetools.convert_to_seconds(timec)
    dep_t_c = nc.Dataset(filesv[-1]).variables['depthv']

    u_u_e, time = analyze.combine_files(filesu, 'vozocrtx','None',[j_e-1, j_e], [i_e-1,i_e])
    v_v_e, timee = analyze.combine_files(filesv, 'vomecrty','None',[j_e-1, j_e], [i_e-1,i_e])
    time_e = tidetools.convert_to_seconds(timee)
    dep_t_e = nc.Dataset(filesv[-1]).variables['depthv']
    
    depth=[dep_t_c[:], dep_t_e[:]] 
    
    u_u_0 = np.ma.masked_values(u_u_e, 0)
    v_v_0 = np.ma.masked_values(v_v_e, 0)
    u_u_0c = np.ma.masked_values(u_u_c, 0)
    v_v_0c = np.ma.masked_values(v_v_c, 0)

    u_c, v_c=research_VENUS.unstag_rot_gridded(u_u_0c, v_v_0c, 'Central')
    u_e, v_e=research_VENUS.unstag_rot_gridded(u_u_0, v_v_0, 'East')
    times=[time_c,time_e]
    us=[u_c, u_e]
    vs=[v_c, v_e]
    i=np.arange(0,2)

    thesize=(40,2)
    vM2amp = np.zeros(thesize); vM2pha = np.zeros(thesize)
    vK1amp = np.zeros(thesize); vK1pha = np.zeros(thesize)
    uM2amp = np.zeros(thesize); uM2pha = np.zeros(thesize)
    uK1amp = np.zeros(thesize); uK1pha = np.zeros(thesize)

    for i, u, time, v in zip(i, us, times, vs):
        uM2amp[:,i], uM2pha[:,i], uK1amp[:,i], uK1pha[:,i] = tidetools.fittit(u, time)
        vM2amp[:,i], vM2pha[:,i], vK1amp[:,i], vK1pha[:,i] = tidetools.fittit(v, time) 

    CX, SX, CY, SY, ap, am, ep, em, major, minor, theta = tidetools.ellipse_params (uM2amp, uM2pha, vM2amp, vM2pha)
    CX_k, SX_k, CY_k, SY_k, ap_k, am_k, ep_k, em_k, major_k, minor_k, theta_k = tidetools.ellipse_params (uK1amp, uK1pha, vK1amp, vK1pha)
    return depth, major, minor, theta, major_k, minor_k, theta_k

In [262]:
def get_filenames(t_orig, t_final, station, model_path):
    """Returns a list with the filenames for all files over the
    defined period of time and sorted in chronological order.

    :arg t_orig: The beginning of the date range of interest.
    :type t_orig: datetime object

    :arg t_final: The end of the date range of interest.
    :type t_final: datetime object

    :arg period: Time interval of model results (eg. 1h or 1d).
    :type period: string

    :arg grid: Type of model results (eg. grid_T, grid_U, etc).
    :type grid: string

    :arg model_path: Defines the path used (eg. nowcast)
    :type model_path: string

    :returns: files, a list of filenames
    """

    numdays = (t_final-t_orig).days
    dates = [t_orig + datetime.timedelta(days=num)
             for num in range(0, numdays+1)]
    dates.sort()
    
    allfiles = glob.glob(model_path+'/*/VENUS_*_gridded.nc')
    sdt = dates[0].strftime('%d%b%y')
    edt = dates[-1].strftime('%d%b%y')
    
    sdt = sdt.lower()
    edt = edt.lower()
    
    sstr = '/{}/VENUS_{}_gridded.nc'.format(sdt, station)
    estr = '/{}/VENUS_{}_gridded.nc'.format(edt, station)
    print estr
    files = []
    for filename in allfiles:
        if os.path.basename(filename) >= sstr:
            if os.path.basename(filename) <= estr:
                files.append(filename)

    files.sort(key=os.path.basename)

    return files

In [263]:
path = '/data/dlatorne/MEOPAR/SalishSea/nowcast'
to=datetime.datetime(2015,5,9)
tf=datetime.datetime(2015,5,11)

filesu=get_filenames(to,tf, 'east', path)
filesv=get_filenames(to,tf,'central', path)
print filesu

/11may15/VENUS_east_gridded.nc
/11may15/VENUS_central_gridded.nc
[]


In [256]:
def loadparam_15(to, tf):
    files = []
    files_East = glob.glob(os.path.join(path,'*','VENUS_east_gridded.nc'))
    for f in files_East:
        directory=os.path.dirname(f)
        files.append(glob.glob(os.path.join(directory,'*_1h_*_grid_T*'))[0])
    files.sort(key=os.path.basename)

    files_East = [];
    files_Central=[];
    for f in files:
        directory=os.path.dirname(f)
        files_East.append(os.path.join(directory,'VENUS_east_gridded.nc'))
        files_Central.append(os.path.join(directory,'VENUS_central_gridded.nc'))
    
    u_u_c, timer = analyze.combine_files(files_Central, 'vozocrtx','None',[0,1], [0,1])
    v_v_c, time = analyze.combine_files(files_Central, 'vomecrty','None', [0,1], [0,1])
    time_c = tidetools.convert_to_seconds(timer)
    dep_t_c, time = combine_t(files_Central, 'depthv')

    u_u_e, time = analyze.combine_files(files_East,'vozocrtx', 'None', [0,1], [0,1])
    v_v_e, time = analyze.combine_files(files_East,'vomecrty', 'None',[0,1], [0,1])
    time_e, time = tidetools.convert_to_seconds(timer)
    dep_t_e, time = combine_t(files_East, 'depthv')
    
    depth=[dep_t_c[:], dep_t_e[:]] 
    
    u_u_0 = np.ma.masked_values(u_u_e, 0)
    v_v_0 = np.ma.masked_values(v_v_e, 0)
    u_u_0c = np.ma.masked_values(u_u_c, 0)
    v_v_0c = np.ma.masked_values(v_v_c, 0)

    u_c, v_c=research_VENUS.unstag_rot_gridded(u_u_0c, v_v_0c, 'Central')
    u_e, v_e=research_VENUS.unstag_rot_gridded(u_u_0, v_v_0, 'East')
    times=[time_c,time_e]
    us=[u_c, u_e]
    vs=[v_c, v_e]
    i=np.arange(0,2)

    thesize=(40,2)
    vM2amp = np.zeros(thesize); vM2pha = np.zeros(thesize)
    vK1amp = np.zeros(thesize); vK1pha = np.zeros(thesize)
    uM2amp = np.zeros(thesize); uM2pha = np.zeros(thesize)
    uK1amp = np.zeros(thesize); uK1pha = np.zeros(thesize)

    for i, u, time, v in zip(i, us, times, vs):
        uM2amp[:,i], uM2pha[:,i], uK1amp[:,i], uK1pha[:,i] = tidetools.fittit(u, time)
        vM2amp[:,i], vM2pha[:,i], vK1amp[:,i], vK1pha[:,i] = tidetools.fittit(v, time) 

    CX, SX, CY, SY, ap, am, ep, em, major, minor, theta = tidetools.ellipse_params (uM2amp, uM2pha, vM2amp, vM2pha)
    CX_k, SX_k, CY_k, SY_k, ap_k, am_k, ep_k, em_k, major_k, minor_k, theta_k = tidetools.ellipse_params (uK1amp, uK1pha, vK1amp, vK1pha)
    return depth, major, minor, theta, major_k, minor_k, theta_k

Function to save tidal parameters in a csv file.

In [185]:
def writetocsv(runname, depth, major, minor,theta, majork1, minork1, thetak1, station):
    outfile = runname+'.csv'
    
    if station == 'Central':
        k=0
    else:
        k=1
    
    with open(outfile, 'wb') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow([
                'Depth', 'Major-Axis (M2)', 'Minor-Axis (M2)', 'Theta (M2)',
                'Major-Axis (K1)', 'Minor-Axis (K1)', 'Theta (K1)'
            ])
        for i in np.arange(0,39):
            writer.writerow([depth[k][i], major[i,k], minor[i,k], theta[i,k], majork1[i,k], minork1[i,k], thetak1[i,k]])


###All the hourly data

In [186]:
path = '/data/dlatorne/MEOPAR/SalishSea/nowcast.nc'
to=datetime.datetime(2014,11,1)
tf=datetime.datetime(2015,6,8)

depth, major, minor, theta, major_k, minor_k, theta_k= loadparam_h(to,tf)

runname = '20141101_20150608_h_Central'
runnamee = '20141101_20150608_h_East'

writetocsv(runname, depth, major, minor, theta, major_k, minor_k, theta_k, 'Central')
writetocsv(runnamee, depth, major, minor, theta, major_k, minor_k, theta_k, 'East')

Then to open and read the columns you could use the lines below:

In [187]:
df = pd.read_csv('20150601_20150608_h_Central.csv')
depth = df.Depth

### The Month of May 2015

In [188]:
to=datetime.datetime(2015,5,1)
tf=datetime.datetime(2015,5,30)

depth, major, minor, theta, major_k, minor_k, theta_k= loadparam_h(to,tf)
runname1 = '20150501_20150530_h_Central'
runname2 = '20150501_20150530_h_East'

writetocsv(runname1, depth, major, minor, theta, major_k, minor_k, theta_k, 'Central')
writetocsv(runname2, depth, major, minor, theta, major_k, minor_k, theta_k, 'East')